# Assignment #2: Language models

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment
* Optionally read a short article on the importance of corpora


## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/corpus/SelmaCorpus.txt


In [3]:
corpus = open("/kaggle/input/corpus/SelmaCorpus.txt","r", encoding='utf-8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [4]:
pattern = 'Nils Holgersson'
width = 25

In [5]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [6]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [7]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [8]:
no_unique_words_orig = len(set(words))
no_unique_words_orig

44266

Lowercased text

In [9]:
lowered = [e.lower() for e in words]
no_unique_words_low = len(set(lowered))
no_unique_words_low

41041

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [10]:
nonletter = '[^\p{L}.;:?!]'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [11]:
def clean(text):
    return re.sub(nonletter, ' ', text)

In [12]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [13]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa. Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [14]:
sentence_boundaries = '([.;:?!])(\s+)(\p{Lu})'
re.findall(sentence_boundaries, test_para)

[('.', ' ', 'H')]

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [15]:
sentence_markup = ' </s>\n<s> \g<3>'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [16]:
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [17]:
print(text)

En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.


The output should look like this:

`En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.`

Insert markup codes in the beginning and end of the text

In [18]:
text = '<s> ' + text + " </s>"
text

'<s> En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>\n<s> Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden. </s>'

The output should look like this:

`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s>`

Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [19]:
text = re.sub('\p{Zs}+', ' ', text)
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


The output should look like this:
    
`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden </s>`

Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [20]:
def segment_sentences(text):
    sentence_boundaries = '([.;:?!])(\p{Zs}+)(\p{Lu})'
    sentence_markup = ' </s>\n<s> \g<3>'
    separated_text = re.sub(sentence_boundaries, sentence_markup, text)
    start_plus_end_text = '<s> ' + separated_text[:-1] + " </s>"
    removed_space_duplicates = re.sub('\p{Zs}+', ' ', start_plus_end_text)
    return removed_space_duplicates.lower()

In [21]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [22]:
corpus = segment_sentences(clean(corpus))

In [23]:
print(corpus[-557:])

s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare. </s>


The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this:

```
<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> 
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>
```

You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [24]:
def create_list(text):
    words = re.findall('[\p{L}<>\/]+', text)
    return words

words = create_list(corpus)

In [25]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


The five last lines of the corpus should like this:

`['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']`



### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [26]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [27]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('users', 1),
 ('deslay', 1),
 ('documents', 1),
 ('spra', 1),
 ('kteknologi', 1),
 ('edan', 2),
 ('notebooks', 1),
 ('selmacorpus', 1)]

#### Bigrams

In [28]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [29]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'users'), 1),
 (('users', 'deslay'), 1),
 (('deslay', 'documents'), 1),
 (('documents', 'spra'), 1),
 (('spra', 'kteknologi'), 1),
 (('kteknologi', 'edan'), 1),
 (('edan', 'edan'), 1),
 (('edan', 'notebooks'), 1)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus. This topic will be discussed during the lab session.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_. 

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [30]:
from functools import reduce

def unigram_lm(freqs, words):
    num_w = 0
    for s in freqs.keys():
        num_w = num_w + freqs[s]
    print("="*50)
    print("wi \t C(wi) \t #words \t P(wi)")
    print("="*50)
    probs_uni = []
    for word in words:
        if word in freqs.keys():
            print(f"{word} \t {freqs[word]} \t {num_w} \t {freqs[word] / num_w}")
            probs_uni.append(freqs[word] / num_w)
        else:
             print(f"{word} \t {0} \t {num_w} \t {0 / num_w}")
    
    multiplied = reduce((lambda x, y: x * y), probs_uni) 
    geo_mean = multiplied**(1/len(probs_uni)) / len(probs_uni)
    entropy_sub = [p*math.log(p,2) for p in probs_uni]
    entropy = -1 * sum(entropy_sub)
    perplexity = (1 / multiplied)**(1/len(probs_uni))
    
    print("="*50)
    print(f"Prob. unigrams: \t {multiplied}")
    print(f"Geometric mean prob.: \t {geo_mean}")
    print(f"Entropy rate: \t {entropy}")
    print(f"Perplexity: \t {perplexity}")
    
    return perplexity

In [31]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [32]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi 	 C(wi) 	 #words 	 P(wi)
det 	 21108 	 1041589 	 0.02026519097263892
var 	 12090 	 1041589 	 0.011607265437711036
en 	 13514 	 1041589 	 0.012974407371813643
gång 	 1332 	 1041589 	 0.001278815348472382
en 	 13514 	 1041589 	 0.012974407371813643
katt 	 16 	 1041589 	 1.536114532699558e-05
som 	 16288 	 1041589 	 0.0156376459428815
hette 	 97 	 1041589 	 9.31269435449107e-05
nils 	 87 	 1041589 	 8.352622771553847e-05
</s> 	 59047 	 1041589 	 0.056689346757694256
Prob. unigrams: 	 5.363621961341534e-27
Geometric mean prob.: 	 0.000236018375074384
Entropy rate: 	 0.6947223837696388
Perplexity: 	 423.6958243970784


In [33]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [34]:
def bigram_lm(freqs, frequency_bigrams, words):
    num_w = sum(freqs.values())
    num_bi = sum(frequency_bigrams.values())
    
    bigrams = [tuple(words[inx:inx + 2])
                   for inx in range(len(words) - 1)]
    
    print("="*50)
    print("wi \t wi+1 \t Ci,i+1   C(i) \t P(wi+1|wi)")
    print("="*50)
    probs_bi = []
    for bigram in bigrams:
        if bigram in frequency_bigrams.keys():
            prob_bayes = (frequency_bigrams[bigram] / num_bi) * (1) / (freqs[bigram[0]]/ num_w)
            probs_bi.append(prob_bayes)
            print(f"{bigram[0]} \t {bigram[1]} \t {frequency_bigrams[bigram]} \t {freqs[bigram[0]]} \t {prob_bayes}")
        else:
            # Stupid backoff
            backoff = 1 * freqs[bigram[1]] / num_w
            probs_bi.append(backoff)
            print(f"{bigram[0]} \t {bigram[1]} \t {0} \t {freqs[bigram[0]]} \t {0.0} \t *backoff: \t {backoff}")
    
    multiplied = reduce((lambda x, y: x * y), probs_bi) 
    geo_mean = multiplied**(1/len(probs_bi)) / len(probs_bi)
    entropy_sub = [p*math.log(p,2) for p in probs_bi]
    entropy = -1 * sum(entropy_sub)
    perplexity = (1 / multiplied)**(1/len(probs_bi))
    
    print("="*50)
    print(f"Prob. unigrams: \t {multiplied}")
    print(f"Geometric mean prob.: \t {geo_mean}")
    print(f"Entropy rate: \t {entropy}")
    print(f"Perplexity: \t {perplexity}")
    
    return perplexity

In [35]:
sent_words.insert(0, '<s>')
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi 	 wi+1 	 Ci,i+1   C(i) 	 P(wi+1|wi)
<s> 	 det 	 5672 	 59047 	 0.09605916381071428
det 	 var 	 3839 	 21108 	 0.18187434554284374
var 	 en 	 712 	 12090 	 0.058891702528670244
en 	 gång 	 706 	 13514 	 0.05224216943992809
gång 	 en 	 20 	 1332 	 0.015015029430518089
en 	 katt 	 6 	 13514 	 0.0004439844428322501
katt 	 som 	 2 	 16 	 0.1250001200090631
som 	 hette 	 45 	 16288 	 0.0027627727899842036
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352622771553847e-05
nils 	 </s> 	 2 	 87 	 0.02298852781775873
Prob. unigrams: 	 2.376124142446303e-19
Geometric mean prob.: 	 0.0013727356507007727
Entropy rate: 	 1.8556370539000757
Perplexity: 	 72.84723752089533


In [36]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In [37]:
test_set = [
    'en liten katt på taket',
    'hur ofta får man rätt första gången',
    'vid den vackra sjön stod en pojke',
    'det är viktigt att komma på rätt sak',
    'detta är ett viktigt fönster'
]

for i, s in enumerate(test_set):
    s = s.split()
    test_p_uni = unigram_lm(frequency, s)
    test_p_bi = bigram_lm(frequency, frequency_bigrams, s)
    print(f"Test sentence {i} : Perplexity unigrams/bigrams: {test_p_uni} / {test_p_bi} ")

wi 	 C(wi) 	 #words 	 P(wi)
en 	 13514 	 1041589 	 0.012974407371813643
liten 	 592 	 1041589 	 0.0005683623770988365
katt 	 16 	 1041589 	 1.536114532699558e-05
på 	 14250 	 1041589 	 0.013681020056855439
taket 	 67 	 1041589 	 6.4324796056794e-05
Prob. unigrams: 	 9.968581117979318e-17
Geometric mean prob.: 	 0.0001261120731547574
Entropy rate: 	 0.17330328670022624
Perplexity: 	 1585.8909856677374
wi 	 wi+1 	 Ci,i+1   C(i) 	 P(wi+1|wi)
en 	 liten 	 419 	 13514 	 0.031004913591118802
liten 	 katt 	 1 	 592 	 0.0016891908109332849
katt 	 på 	 0 	 16 	 0.0 	 *backoff: 	 0.013681020056855439
på 	 taket 	 19 	 14250 	 0.001333334613430006
Prob. unigrams: 	 9.55359592762272e-10
Geometric mean prob.: 	 0.001389894146315906
Entropy rate: 	 0.26837612738348654
Perplexity: 	 179.86981286499932
Test sentence 0 : Perplexity unigrams/bigrams: 1585.8909856677374 / 179.86981286499932 
wi 	 C(wi) 	 #words 	 P(wi)
hur 	 1996 	 1041589 	 0.0019163028795426988
ofta 	 144 	 1041589 	 0.0001382503079429

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program it following the model of bigrams.

In [38]:
def trigrams(words):
    trigrams = []
    words = [ e for e in words if e != '<s>' and e != '</s>']
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))
    frequency_trigrams = {}
    for i in range(len(trigrams)):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [39]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [40]:
starting_text = 'De'.lower()

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies.

In [41]:
cand_nbr = 5

In [42]:
def candidates_bi(starting_text, cand_nbr):
    candidates = {}
    num_bi = sum(frequency_bigrams.values())
    
    for bigram in frequency_bigrams.keys():
        
        if bigram[0] == "<s>":
            if bigram[1].startswith(starting_text.lower()):
                prob_bigram = frequency_bigrams[bigram] / num_bi

                if len(candidates) < cand_nbr:
                    if bigram[1] not in [bi[1] for bi in candidates.keys()]:
                        candidates[bigram] = prob_bigram
                else:
                    key_min = min(candidates, key=candidates.get) # get min value in dict
                    if prob_bigram > candidates[key_min]:
                        if bigram[1] not in [bi[1] for bi in candidates.keys()]:
                            del candidates[key_min]
                            candidates[bigram] = prob_bigram
                
    candidates_sorted = {k: v for k, v in sorted(candidates.items(), key=lambda item: item[1], reverse=True)}
    print(candidates_sorted)
    return list(candidates_sorted.keys())

current_word_predictions_1 = [c[1] for c in candidates_bi(starting_text, cand_nbr)]

{('<s>', 'det'): 0.005445531246519737, ('<s>', 'de'): 0.0019883101571830705, ('<s>', 'den'): 0.0013200996939288854, ('<s>', 'detta'): 0.0002861016063933148, ('<s>', 'denna'): 7.680580037404424e-05}


In [43]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [44]:
current_text = "Det var en ".lower()

Tokenize this text and return a list of tokens. Call it `tokens`.

In [45]:
def tokenize_with_space(text):
    words = re.findall('\p{L}+', text)
    if text.endswith(' '):
        words.append(' ')
    return words
tokens = tokenize_with_space(current_text)

In [46]:
tokens

['det', 'var', 'en', ' ']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`

In [47]:
def candidates_tri(starting_text, cand_nbr, space):
    candidates = {}
    num_tri = sum(frequency_trigrams.values())
    
    for trigram in frequency_trigrams.keys():
        if (space == True and starting_text[-3:-1] == list(trigram)[0:2]) or (space == False and starting_text[-3:-1] == list(trigram)[0:2] and trigram[2].startswith(starting_text[-1])):
            prob_trigram = frequency_trigrams[trigram] / num_tri
            if len(candidates) < cand_nbr:
                candidates[trigram] = prob_trigram
            else:
                key_min = min(candidates, key=candidates.get) # get min value in dict
                if prob_trigram > candidates[key_min]:
                    del candidates[key_min]
                    candidates[trigram] = prob_trigram
                
    candidates_sorted = {k: v for k, v in sorted(candidates.items(), key=lambda item: item[1], reverse=True)}
    print(candidates_sorted)
    return list(candidates_sorted.keys())

def propose_next_five(words):
    space = False
    if words[-1] == ' ':
        space = True
    else:
        space = False
    candidates1 = candidates_tri(words, 5, space)
    # add two more words from last one in proposed trigrams.
    #candidates2 = [candidates_tri(c[2], 1)[0] for c in candidates1]
    #five_candidates = [(c1 + c2[1:3]) for c1, c2 in zip(candidates1, candidates2)] 
    #return five_candidates
    return candidates1

In [48]:
five_next_trigrams = propose_next_five(tokens)
next_word_predictions = [c[2] for c in five_next_trigrams]

{('var', 'en', 'stor'): 3.6816738188594827e-05, ('var', 'en', 'liten'): 2.8153976261866632e-05, ('var', 'en', 'gammal'): 2.057405957597946e-05, ('var', 'en', 'god'): 1.8408369094297413e-05, ('var', 'en', 'sådan'): 1.8408369094297413e-05}


In [49]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [50]:
current_text = "Det var en g".lower()

In [51]:
current_word_predictions_2 = [c[2] for c in propose_next_five(tokenize(current_text))]
#current_word_predictions_2 = propose_next_five(tokenize(current_text))

{('var', 'en', 'gammal'): 2.057405957597946e-05, ('var', 'en', 'god'): 1.8408369094297413e-05, ('var', 'en', 'gång'): 1.0828452408410243e-05, ('var', 'en', 'ganska'): 3.2485357225230726e-06, ('var', 'en', 'grann'): 2.1656904816820487e-06}


In [52]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'grann']

## Checked answers

The system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

The submission code will send your answer. It consists of the perplexities and predictions.

In [53]:
ANSWER = str((perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2))
ANSWER

"(423, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'grann'])"

## Reading

<p>As an application of n-grams, execute the Jupyter notebook by Peter Norvig <a
        href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">
    here</a>. Just run all the cells and be sure that you understand the code.
    You will find the data <a href="http://norvig.com/ngrams/">here</a>.</p>
<p>In your report, you will also describe one experiment with a long string of words
    your will create yourself or copy from a text you like. You will remove all the punctuation and
    white spaces from this string. Set this string in lowercase letters.</p>
<p>You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and
    run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. </p>
<p>You will comment the segmentation results you obtain with unigram and bigram models.
</p>